### Win Rate

Win rate of player and opponent

In [1]:
import pandas as pd

games_df = pd.read_csv("../../data/processed/data.csv")

games_df.sort_values(by=['date'], inplace=True, ignore_index=True)
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0
...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5


In [2]:
def win_rate(df, player):
    df['result'] = df['win_name'].apply(lambda x: 1 if player == x else 0)

    df.insert(df.shape[1], "num_games", range(1, len(df)+1))
    df['cum_sum'] = df['result'].cumsum()

    df['win_rate'] = df['cum_sum']/df['num_games']
    df['win_rate'] = df['win_rate'].shift()
    
    df = df.apply(assign_win_rate, player=player, axis=1)
    return df

def assign_win_rate(x, player):
    if player == x['win_name']:
        x['PWR'] = x['win_rate']
    else:
        x['OWR'] = x['win_rate']
    return x

In [3]:
from tqdm import tqdm

games_df['PWR'], games_df['OWR'] = 0, 0

for player in tqdm(games_df.win_name.unique()):
    filtered_df = games_df.loc[
        (games_df.win_name == player) | (games_df.lose_name == player)
    ].copy()

    df = win_rate(filtered_df, player)
    games_df.update(df)

games_df

100%|██████████| 1827/1827 [02:07<00:00, 14.31it/s]


,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi,PWR,OWR
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0,0.000000,0.000000
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0,0.000000,1.000000
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0,0.500000,0.500000
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0,0.333333,0.666667
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...
101404,Jin Siyoung,Choi Kyeongho,w,b,2022-02-19,8p,5p,6.5,0.482759,0.418919
101405,Kang Dongyun,Seol Hyunjun,b,w,2022-02-19,9p,7p,6.5,0.587283,0.518868
101406,Kim Jiseok,Shin Jaeweon,w,b,2022-02-19,9p,4p,6.5,0.628788,0.444444
101407,Kim Changhoon,Kwak Wonkeun,b,w,2022-02-19,5p,3p,6.5,0.640000,0.571429


In [4]:
games_df.loc[(games_df['win_name'] == 'Lee Changseok') | (games_df['lose_name'] == 'Lee Changseok')]

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi,PWR,OWR
56557,Gong Yanyu,Lee Changseok,w,b,2013-07-09,2p,5d,7.5,0.444444,0.000000
64061,Lee Changseok,You Byungyong,b,w,2015-04-25,1p,2p,6.5,0.000000,0.250000
64183,Lee Changseok,Oh Jangwuk,b,w,2015-05-03,1p,2p,6.5,0.500000,0.400000
64341,Lee Changseok,Lee Jihyun (m),b,w,2015-05-10,1p,4p,6.5,0.666667,0.540984
64390,Lee Changseok,Park Kyuongkeun,w,b,2015-05-13,1p,3p,6.5,0.750000,0.222222
...,...,...,...,...,...,...,...,...,...,...
100834,Lee Changseok,Cho Wankyu,w,b,2022-01-07,8p,3p,6.5,0.556054,0.352941
101008,Park Sangjin,Lee Changseok,b,w,2022-01-14,5p,8p,6.5,0.507937,0.558036
101175,Lee Changseok,Han Sanghoon,b,w,2022-01-23,8p,9p,6.5,0.555556,0.475248
101228,Byun Sangil,Lee Changseok,w,b,2022-02-03,9p,8p,6.5,0.609218,0.557522
